In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

gene_set_x_sample = pd.read_csv(PATH["gene_set_x_sample.tsv"], sep="\t", index_col=0)

In [ ]:
figure_name_data_name_indices = {"Summary": {"Peek": SETTING["gene_sets_to_peek"]}}

In [ ]:
for target_name, target_values in target_x_sample.iterrows():

    target_values = target_values[target_values != -1]

    if target_values.value_counts().min() < 2:

        continue

    output_directory_path = os.path.join(PATH["summarize_gene_set/"], target_name)

    ccal.establish_path(output_directory_path, "directory")

    for figure_name, data_name_indices in figure_name_data_name_indices.items():

        data_dicts = {}

        score_moe_p_value_fdr_dicts = {}

        for data_name, indices in data_name_indices.items():

            data_dicts[data_name] = {
                "dataframe": gene_set_x_sample.loc[indices],
                "type": "continuous",
            }

            score_moe_p_value_fdr_dicts[data_name] = pd.read_csv(
                os.path.join(
                    PATH["find_differentially_expressed_gene_set/"],
                    target_name,
                    "all.tsv",
                ),
                sep="\t",
                index_col=0,
            )

        ccal.make_summary_match_panel(
            target_values,
            data_dicts,
            score_moe_p_value_fdr_dicts,
            target_type="binary",
            plot_std=SETTING["plot_std"],
            title={"text": figure_name},
            html_file_path=os.path.join(
                output_directory_path, "{}.html".format(figure_name)
            ),
        )